# MapVec end-to-end pipeline 📒

This notebook runs the **entire pipeline**:
1. Prompt embeddings (Universal Sentence Encoder)
2. Map embeddings (handcrafted polygon features)
3. Concatenation into a training matrix
4. Helper cells to inspect vectors by `prompt_id` or `map_id`

**Edit the Parameters** in the next cell to match your project layout.


In [20]:
# ===================== PARAMETERS =====================
from pathlib import Path
import sys
import subprocess
import importlib
import shutil
import numpy as np
import pandas as pd
import shlex
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, mean_squared_error, mean_absolute_error
from sklearn.utils.class_weight import compute_class_weight
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

PROJ_ROOT = Path("../").resolve()       # adjust if your notebook sits elsewhere
SRC_DIR   = PROJ_ROOT / "src"
# Put project root on sys.path
if str(PROJ_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJ_ROOT))

from src.config import PATHS, CFG, print_summary

In [21]:
print_summary()  # optional

# Access paths like:
MAPS_ROOT = PATHS.MAPS_ROOT
INPUT_MAPS_PATTERN = PATHS.INPUT_MAPS_PATTERN
PROMPTS_CSV = PATHS.PROMPTS_CSV
PRM_NPZ = PATHS.PRM_NPZ

# Dims (auto-inferred if available; else None until you set them)
MAP_DIM = CFG.MAP_DIM or 996         # fallback
PROMPT_DIM = CFG.PROMPT_DIM or 512   # fallback
FUSED_DIM = CFG.FUSED_DIM or (MAP_DIM + PROMPT_DIM)
BATCH_SIZE = CFG.BATCH_SIZE

=== CONFIG SUMMARY ===
PROJ_ROOT  : /Users/amirdonyadide/Documents/GitHub/Thesis
DATA_DIR   : /Users/amirdonyadide/Documents/GitHub/Thesis/data
INPUT_DIR  : /Users/amirdonyadide/Documents/GitHub/Thesis/data/input
OUTPUT_DIR : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output
MAPS_ROOT  : /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/samples/pairs
INPUT PAT. : *_input.geojson
PROMPTS_CSV: /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/prompts.csv
PAIRS_CSV  : /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/pairs.csv
PROMPT_OUT : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out
MAP_OUT    : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out
TRAIN_OUT  : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out
MODEL_OUT  : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/models
SPLIT_OUT  : /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out/splits
PRM_NPZ    : /Users/amirdonyadide/Document

In [22]:
PATHS.clean_outputs()

🧹 Removing old directory: /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out
🧹 Removing old directory: /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out
🧹 Removing old directory: /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out
🧹 Removing old directory: /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/models
✅ All output folders cleaned and recreated fresh.



## 1) Prompt embeddings
Runs `src/mapvec/prompts/prompt_embeddings.py` using your chosen USE model and saves artifacts to `PROMPT_OUT`.

In [29]:
cmd = [
    sys.executable, "-m", "src.mapvec.prompts.prompt_embeddings",
    "--input",    str(PATHS.PROMPTS_CSV),
    "--model",    str(CFG.USE_MODEL),
    "--l2",       
    "--out_dir",  str(PATHS.PROMPT_OUT),
    "-v"
]
print("CMD:", " ".join(cmd))
res = subprocess.run(cmd, cwd=str(PATHS.PROJ_ROOT))  # shell=False by default
if res.returncode != 0:
    raise SystemExit("Prompt embedding step failed.")
print("Prompt embeddings completed.")

CMD: /opt/anaconda3/envs/thesis/bin/python -m src.mapvec.prompts.prompt_embeddings --input /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/prompts.csv --model dan --l2 --out_dir /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out -v


15:59:39 | DEBUG | FILE_DIR=/Users/amirdonyadide/Documents/GitHub/Thesis/src/mapvec/prompts
15:59:39 | DEBUG | PROJECT_ROOT=/Users/amirdonyadide/Documents/GitHub/Thesis
15:59:39 | DEBUG | DEFAULT_DATA_DIR=/Users/amirdonyadide/Documents/GitHub/Thesis/data
15:59:39 | INFO | DATA_DIR=/Users/amirdonyadide/Documents/GitHub/Thesis/data
15:59:39 | INFO | INPUT=/Users/amirdonyadide/Documents/GitHub/Thesis/data/input/prompts.csv
15:59:39 | INFO | OUT_DIR=/Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out
15:59:39 | INFO | Reading CSV: /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/prompts.csv
15:59:39 | INFO | Loaded 500 prompts (id_col=prompt_id). Sample IDs: p001, p002, p003…
15:59:39 | INFO | Using local USE-dan at /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/model_dan
15:59:39 | INFO | Loading USE-dan from local path: /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/model_dan …
15:59:42 | INFO | Fingerprint not found. Saved model loading will cont

Prompt embeddings completed.


## 2) Map embeddings
Runs the map embedding module on the GeoJSON inputs. Skips problematic features, logs warnings, and writes `embeddings.npz` to `PAIR_MAP_OUT`.

In [24]:
cmd = [
    sys.executable, "-m", "src.mapvec.maps.map_embeddings",
    "--root", str(PATHS.MAPS_ROOT),
    "--pattern", PATHS.INPUT_MAPS_PATTERN,
    "--out_dir", str(PATHS.MAP_OUT),
    "-v"
]
print("CMD:", " ".join(cmd))
res = subprocess.run(cmd, cwd=str(PATHS.PROJ_ROOT)) 
if res.returncode != 0:
    raise SystemExit("Map embedding step failed.")
print("✅ Input map embeddings completed.")

CMD: /opt/anaconda3/envs/thesis/bin/python -m src.mapvec.maps.map_embeddings --root /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/samples/pairs --pattern *_input.geojson --out_dir /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out -v


15:42:14 | DEBUG | PROJECT_ROOT=/Users/amirdonyadide/Documents/GitHub/Thesis
15:42:14 | DEBUG | DATA_DIR=/Users/amirdonyadide/Documents/GitHub/Thesis/data
15:42:14 | INFO | Scanning /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/samples/pairs (pattern=*_input.geojson)…
15:42:15 | INFO | OK  map_id=0073  -> vector[249]
15:42:17 | INFO | OK  map_id=0080  -> vector[249]
15:42:17 | INFO | OK  map_id=0093  -> vector[249]
15:42:21 | INFO | OK  map_id=0122  -> vector[249]
15:42:22 | INFO | OK  map_id=0123  -> vector[249]
15:42:23 | INFO | OK  map_id=0127  -> vector[249]
15:42:24 | INFO | OK  map_id=0158  -> vector[249]
15:42:26 | INFO | OK  map_id=0159  -> vector[249]
15:42:27 | INFO | OK  map_id=0160  -> vector[249]
15:42:28 | INFO | OK  map_id=0165  -> vector[249]
15:42:29 | INFO | OK  map_id=0167  -> vector[249]
15:42:30 | INFO | OK  map_id=0168  -> vector[249]
15:42:31 | INFO | OK  map_id=0171  -> vector[249]
15:42:34 | INFO | OK  map_id=0208  -> vector[249]
15:42:38 | INFO | OK 

✅ Input map embeddings completed.


15:49:57 | INFO | OK  map_id=1757  -> vector[249]
15:49:57 | INFO | Saved 300 vectors (failed=0) to /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out


## 3) Concatenate
Joins map & prompt vectors using `pairs.csv` and writes `X_concat.npy` and `train_pairs.parquet` to `TRAIN_OUT`.

In [36]:
cmd = [
    sys.executable, "-m", "src.mapvec.concat.concat_embeddings",
    "--pairs",      str(PATHS.PAIRS_CSV),
    "--map_npz",    str(PATHS.MAP_OUT / "maps_embeddings.npz"),   # single-map embeddings
    "--prompt_npz", str(PATHS.PROMPT_OUT / "prompts_embeddings.npz"),
    "--out_dir",    str(PATHS.TRAIN_OUT),
    "--drop_dupes",          # optional flag (keep if you want)
    # "--fail_on_missing",   # optional: uncomment if you prefer hard failure on missing IDs
]

print("CMD:", " ".join(cmd))
res = subprocess.run(cmd, cwd=str(PATHS.PROJ_ROOT))  # run from project root
if res.returncode != 0:
    raise SystemExit("Concatenation step failed.")
print("✅ Concatenation completed successfully.")

CMD: /opt/anaconda3/envs/thesis/bin/python -m src.mapvec.concat.concat_embeddings --pairs /Users/amirdonyadide/Documents/GitHub/Thesis/data/input/pairs.csv --map_npz /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out/maps_embeddings.npz --prompt_npz /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out/prompts_embeddings.npz --out_dir /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out --drop_dupes
✅ Concatenation completed successfully.


16:15:14 | INFO | Map  embeddings: (300, 249) from /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/map_out/maps_embeddings.npz
16:15:14 | INFO | Prompt embeddings: (500, 512) from /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/prompt_out/prompts_embeddings.npz
16:15:14 | INFO | X shape = (450, 761)  (map_dim=249, prompt_dim=512)
16:15:14 | INFO | Saved to /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out in 0.02s


## 4) Split dataset  
Splits the concatenated feature matrix `X_concat.npy` and its metadata `train_pairs.parquet` into separate **training**, **validation**, and **test** subsets.  
Each split preserves row alignment between features and metadata, and the resulting files are saved under `TRAIN_OUT/splits/` as:  

- `X_train.npy`, `pairs_train.parquet`  
- `X_val.npy`, `pairs_val.parquet`  
- `X_test.npy`, `pairs_test.parquet`  

In [37]:
# Load data
X = np.load(PATHS.TRAIN_OUT / "X_concat.npy")
pairs_df = pd.read_parquet(PATHS.TRAIN_OUT / "train_pairs.parquet")

print(f"Loaded X: {X.shape}, pairs: {pairs_df.shape}")

# --- Step 1: Train/Test split
X_train, X_temp, df_train, df_temp = train_test_split(
    X, pairs_df, test_size= CFG.VAL_RATIO + CFG.TEST_RATIO, random_state= CFG.SEED, shuffle=True
)

# --- Step 2: Split temp into Val/Test
relative_test_ratio = CFG.TEST_RATIO / (CFG.VAL_RATIO + CFG.TEST_RATIO)
X_val, X_test, df_val, df_test = train_test_split(
    X_temp, df_temp, test_size=relative_test_ratio, random_state= CFG.SEED, shuffle=True
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# --- Save splits
split_dir = PATHS.TRAIN_OUT / "splits"
split_dir.mkdir(exist_ok=True)

np.save(split_dir / "X_train.npy", X_train)
np.save(split_dir / "X_val.npy",   X_val)
np.save(split_dir / "X_test.npy",  X_test)

df_train.to_parquet(split_dir / "pairs_train.parquet", index=False)
df_val.to_parquet(split_dir / "pairs_val.parquet", index=False)
df_test.to_parquet(split_dir / "pairs_test.parquet", index=False)

print(f"Saved splits to {split_dir}")

Loaded X: (450, 761), pairs: (450, 4)
Train: (315, 761), Val: (67, 761), Test: (68, 761)
Saved splits to /Users/amirdonyadide/Documents/GitHub/Thesis/data/output/train_out/splits


In [38]:
# ===================== MODEL TRAINING PARAMETERS =====================
# --------- CONFIG ----------
OP_COL         = "operator"
PARAM_COLS     = ["param"]                     # keep list so y has shape (n,1)
FIXED_CLASSES  = ["simplify", "select", "aggregate", "displace"]
CLIP_ABS = None  # set to 8.0 to enable, or None to disable

In [39]:
def filter_X_and_df(X, df):
    """Filter rows that have operator and param; normalize operator; keep X aligned."""
    df2 = df.copy()
    df2[OP_COL] = df2[OP_COL].astype(str).str.strip().str.lower()
    mask = df2[OP_COL].notna()
    for c in PARAM_COLS:
        mask &= df2[c].notna()
    mask = mask.values  # numpy bool array aligned with X
    X2   = X[mask]
    df2  = df2.loc[mask].reset_index(drop=True)
    return X2.astype("float32", copy=False), df2

# Apply to your splits produced earlier
X_train, df_train = filter_X_and_df(X_train, df_train)
X_val,   df_val   = filter_X_and_df(X_val,   df_val)
X_test,  df_test  = filter_X_and_df(X_test,  df_test)

In [40]:
# Fixed class order (even if a class is missing in train)
le = LabelEncoder()
le.fit(FIXED_CLASSES)
classes = le.classes_.tolist()
print("Classes:", classes)

y_train_cls = le.transform(df_train[OP_COL])
y_val_cls   = le.transform(df_val[OP_COL])
y_test_cls  = le.transform(df_test[OP_COL])

# Single-parameter targets -> 2D arrays (n,1)
y_train_reg = df_train[PARAM_COLS].to_numpy(dtype="float32")
y_val_reg   = df_val[PARAM_COLS].to_numpy(dtype="float32")
y_test_reg  = df_test[PARAM_COLS].to_numpy(dtype="float32")

Classes: ['aggregate', 'displace', 'select', 'simplify']


In [41]:
# Replace inf with NaN so imputer can handle them
for X in (X_train, X_val, X_test):
    X[~np.isfinite(X)] = np.nan

imp = SimpleImputer(strategy="median")
X_train_imp = imp.fit_transform(X_train)
X_val_imp   = imp.transform(X_val)
X_test_imp  = imp.transform(X_test)

scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(5, 95))
X_train_s = scaler.fit_transform(X_train_imp)
X_val_s   = scaler.transform(X_val_imp)
X_test_s  = scaler.transform(X_test_imp)

# Clip extremes to stabilize optimizer
if CLIP_ABS is not None:
    X_train_s = np.clip(X_train_s, -CLIP_ABS, CLIP_ABS)
    X_val_s   = np.clip(X_val_s,   -CLIP_ABS, CLIP_ABS)
    X_test_s  = np.clip(X_test_s,  -CLIP_ABS, CLIP_ABS)

assert np.isfinite(X_train_s).all() and np.isfinite(X_val_s).all() and np.isfinite(X_test_s).all(), "Non-finite after scaling."

In [44]:
# Class weights for imbalance
cls_w    = compute_class_weight("balanced", classes=np.arange(len(classes)), y=y_train_cls)
sample_w = np.array([cls_w[c] for c in y_train_cls], dtype="float32")
print("Class weights:", dict(zip(classes, cls_w)))

clf = MLPClassifier(
    hidden_layer_sizes=(256, 128),
    activation="relu",
    solver="adam",
    alpha=1e-3,
    learning_rate_init=1e-4,
    batch_size=128,
    max_iter=400,
    early_stopping=True,
    n_iter_no_change=20,
    random_state=CFG.SEED,
    verbose=True
)

try:
    clf.fit(X_train_s, y_train_cls, sample_weight=sample_w)
except Exception as e:
    print(f"⚠️ Adam crashed ({e}). Falling back to lbfgs (no sample_weight).")
    clf = MLPClassifier(
        hidden_layer_sizes=(256, 128),
        activation="relu",
        solver="lbfgs",
        alpha=1e-3,
        max_iter=500,
        random_state= CFG.SEED,
        verbose=True
    )
    clf.fit(X_train_s, y_train_cls)

Class weights: {'aggregate': np.float64(0.9375), 'displace': np.float64(0.9603658536585366), 'select': np.float64(1.0227272727272727), 'simplify': np.float64(1.09375)}
Iteration 1, loss = nan
Validation score: 0.282862
Iteration 2, loss = nan
Validation score: 0.232634
Iteration 3, loss = nan
Validation score: 0.232634
Iteration 4, loss = nan
Validation score: 0.232634
Iteration 5, loss = nan
Validation score: 0.232634
Iteration 6, loss = nan
Validation score: 0.232634
Iteration 7, loss = nan
Validation score: 0.232634
Iteration 8, loss = nan
Validation score: 0.232634
Iteration 9, loss = nan
Validation score: 0.232634
Iteration 10, loss = nan
Validation score: 0.232634
Iteration 11, loss = nan
Validation score: 0.232634
Iteration 12, loss = nan
Validation score: 0.232634
Iteration 13, loss = nan
Validation score: 0.232634
Iteration 14, loss = nan
Validation score: 0.232634
Iteration 15, loss = nan
Validation score: 0.232634
Iteration 16, loss = nan
Validation score: 0.232634
Iteration

/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_stochastic_optimizers.py:275: RuntimeWarning: overflow encountered in square
  self.beta_2 * v + (1 - self.beta_2) * (grad**2)
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:173: RuntimeWarning: invalid value encountered in add
  activations[i + 1] += self.intercepts_[i]
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:173: RuntimeWarning: invalid value encountered in add
  activations[i + 1] += self.intercepts_[i]
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:602: ConvergenceWarning: lbfgs failed to converge after 0 iteration(s) (status=2):
ABNORMAL: 

You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [72]:
def check_matrix(name, X):
    print(f"--- {name} ---")
    print("shape:", X.shape, "dtype:", X.dtype)
    print("finite:", np.isfinite(X).all())
    print("min/max:", np.nanmin(X), np.nanmax(X))
    print("mean/std:", np.nanmean(X), np.nanstd(X))
    # any all-NaN or all-constant columns before scaling?
    col_nan = np.isnan(X).all(axis=0).sum()
    col_zero_var = (np.nanstd(X, axis=0) == 0).sum()
    print("all-NaN cols:", col_nan, "zero-variance cols:", col_zero_var)

check_matrix("X_train", X_train)
check_matrix("X_val",   X_val)
check_matrix("X_test",  X_test)

check_matrix("X_train_imp", X_train_imp)
check_matrix("X_train_s",   X_train_s)

print("y_train_cls has NaN?", np.isnan(y_train_cls).any() if hasattr(y_train_cls, "__len__") else False)
print("classes present in train:", sorted(set(df_train[OP_COL])))

--- X_train ---
shape: (315, 1508) dtype: float32
finite: True
min/max: -552.0 4054873900.0
mean/std: 24875.152 8824922.0
all-NaN cols: 0 zero-variance cols: 58
--- X_val ---
shape: (67, 1508) dtype: float32
finite: True
min/max: -505.0 96049.9
mean/std: 22.502472 675.2408
all-NaN cols: 0 zero-variance cols: 72
--- X_test ---
shape: (68, 1508) dtype: float32
finite: True
min/max: -552.0 4054873900.0
mean/std: 115146.09 18993522.0
all-NaN cols: 0 zero-variance cols: 67
--- X_train_imp ---
shape: (315, 1508) dtype: float32
finite: True
min/max: -552.0 4054873900.0
mean/std: 24875.152 8824922.0
all-NaN cols: 0 zero-variance cols: 58
--- X_train_s ---
shape: (315, 1508) dtype: float32
finite: True
min/max: -20.723267 142484560.0
mean/std: 1179.2317 357503.12
all-NaN cols: 0 zero-variance cols: 58
y_train_cls has NaN? False
classes present in train: ['aggregate', 'displace', 'select', 'simplify']


In [57]:
y_val_pred  = clf.predict(X_val_s)
val_acc     = accuracy_score(y_val_cls, y_val_pred)
val_f1m     = f1_score(y_val_cls, y_val_pred, average="macro")
print(f"[VAL] acc={val_acc:.3f}  f1_macro={val_f1m:.3f}")
print(classification_report(y_val_cls, y_val_pred, target_names=classes))

[VAL] acc=0.224  f1_macro=0.191
              precision    recall  f1-score   support

   aggregate       0.00      0.00      0.00        17
    displace       0.42      0.42      0.42        12
      select       0.50      0.04      0.08        24
    simplify       0.17      0.64      0.27        14

    accuracy                           0.22        67
   macro avg       0.27      0.28      0.19        67
weighted avg       0.29      0.22      0.16        67



/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result

In [58]:
regressors = {}
for idx, name in enumerate(classes):
    sel = (y_train_cls == idx)
    n = int(sel.sum())
    if n < 5:
        print(f"⚠️  Skipping regressor for '{name}' (only {n} samples).")
        continue

    reg = MLPRegressor(
        hidden_layer_sizes=(128, 64),
        activation="relu",
        solver="adam",
        alpha=1e-3,
        learning_rate_init=5e-4,
        max_iter=600,
        early_stopping=True,
        n_iter_no_change=30,
        random_state=SEED,
        verbose=False
    )
    reg.fit(X_train_s[sel], y_train_reg[sel])  # (n,d) -> (n,1)
    regressors[name] = reg
    print(f"✅ Regressor trained for '{name}' on {n} samples.")

/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1771: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:214: RuntimeWarning: invalid value encountered in add
  activation += self.intercepts_[i]
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1771: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ra

✅ Regressor trained for 'aggregate' on 84 samples.


/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1771: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


✅ Regressor trained for 'displace' on 82 samples.


/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1771: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


✅ Regressor trained for 'select' on 77 samples.
✅ Regressor trained for 'simplify' on 72 samples.


/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(


In [59]:
# Classification
y_test_pred = clf.predict(X_test_s)
test_acc    = accuracy_score(y_test_cls, y_test_pred)
test_f1m    = f1_score(y_test_cls, y_test_pred, average="macro")
print(f"[TEST] acc={test_acc:.3f}  f1_macro={test_f1m:.3f}")
print(classification_report(y_test_cls, y_test_pred, target_names=classes))

# Parameter regression (conditioned on correct operator)
mask = (y_test_pred == y_test_cls)
print(f"Parameter evaluation on {int(mask.sum())}/{len(mask)} samples with correct operator prediction.")
y_pred_params = np.full_like(y_test_reg, np.nan, dtype="float32")  # (n,1)

for i, ok in enumerate(mask):
    if not ok:
        continue
    cls_name = classes[y_test_pred[i]]
    reg = regressors.get(cls_name)
    if reg is None:
        continue
    pred = reg.predict(X_test_s[i:i+1])[0]
    y_pred_params[i] = pred if hasattr(pred, "__len__") else [float(pred)]

valid = np.isfinite(y_pred_params).all(axis=1) & mask
if valid.any():
    mse = mean_squared_error(y_test_reg[valid], y_pred_params[valid])
    mae = mean_absolute_error(y_test_reg[valid], y_pred_params[valid])
    print(f"[TEST Param | correct-ops] MSE={mse:.4f}  MAE={mae:.4f}")
else:
    print("No valid parameter predictions to score.")

[TEST] acc=0.279  f1_macro=0.214
              precision    recall  f1-score   support

   aggregate       0.00      0.00      0.00        20
    displace       0.42      0.33      0.37        15
      select       1.00      0.07      0.13        14
    simplify       0.24      0.68      0.35        19

    accuracy                           0.28        68
   macro avg       0.41      0.27      0.21        68
weighted avg       0.36      0.28      0.21        68

Parameter evaluation on 19/68 samples with correct operator prediction.
[TEST Param | correct-ops] MSE=390.9869  MAE=7.3307


/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result

In [60]:
joblib.dump(imp,        MODEL_OUT / "imputer.joblib")
joblib.dump(scaler,     MODEL_OUT / "scaler.joblib")
joblib.dump(le,         MODEL_OUT / "label_encoder.joblib")
joblib.dump(clf,        MODEL_OUT / "mlp_classifier.joblib")
joblib.dump(regressors, MODEL_OUT / "per_class_regressors.joblib")
print(f"Models saved to {MODEL_OUT}")

Models saved to ../data/output/models


In [61]:
def predict_pipeline(X_batch):
    """
    X_batch: (n, 1508) raw concatenated features
    Returns: dict with operator_probs, operator_label, params_pred
    """
    Xb = np.asarray(X_batch, dtype="float32")
    Xb[~np.isfinite(Xb)] = np.nan
    Xb = imp.transform(Xb)
    Xb = scaler.transform(Xb)
    Xb = np.clip(Xb, -CLIP_ABS, CLIP_ABS)

    proba = clf.predict_proba(Xb)                    # (n, 4)
    pred_idx = np.argmax(proba, axis=1)
    pred_labels = le.inverse_transform(pred_idx).tolist()

    params = []
    for i, lbl in enumerate(pred_labels):
        reg = regressors.get(lbl)
        if reg is None:
            params.append([np.nan])                 # single param
        else:
            pred = reg.predict(Xb[i:i+1])[0]
            params.append(pred.tolist() if hasattr(pred, "tolist") else [float(pred)])

    return {"operator_probs": proba, "operator_label": pred_labels, "params_pred": np.array(params)}